In [48]:
import read_write_file as rwf
import numpy as np

Задание с лекции. Непрапвильный One-Time Pad

In [49]:
from operator import xor

def create_ascii_byte_array():
    ascii_bytes = []

    # Добавляем коды заглавных букв A-Z
    for i in range(65, 91):  # ASCII коды от 65 до 90
        ascii_bytes.append(i)

    # Добавляем коды строчных букв a-z
    for i in range(97, 123):  # ASCII коды от 97 до 122
        ascii_bytes.append(i)

    # Добавляем код пробела
    ascii_bytes.append(32)  # ASCII код пробела

    # Добавляем код запятой
    ascii_bytes.append(44)  # ASCII код запятой

    # Добавляем код точки
    ascii_bytes.append(46)  # ASCII код точки

    return ascii_bytes


def read_hex_data(file_path):
    with open(file_path, 'r') as file:
        data = file.read()
        hex_array = data.split()
        return hex_array

def one_time_pad_wrong():
    p_i = 0
    l = 256
    ascii_bytes = create_ascii_byte_array()
    encrypt_data_arr = read_hex_data('one_time_pad_wrong.txt')
    hex_array = [int(x, 16) for x in encrypt_data_arr]
    decrypt_data_arr = []
    for c_i in hex_array:
        binary_representation = bin(p_i)[2:].zfill(8)
        reversed_binary = binary_representation[::-1]
        reversed_p = int(reversed_binary, 2)
        print(binary_representation, reversed_binary, reversed_p)
        p_i_1 = reversed_p % l
        p_i_2 = (reversed_p + 0x5A) % l
        print(p_i_1, p_i_2, c_i)
        c_i_bin = bin(c_i)[2:].zfill(8)
        c_i_bin_int = int(c_i_bin, 2)
        m_i_1 = xor(c_i_bin_int, p_i_1) % l
        m_i_2 = xor(c_i_bin_int, p_i_2) % l
        print(m_i_1, m_i_2, type(m_i_1), c_i)
        if m_i_1 in ascii_bytes:
            decrypt_data_arr.append(m_i_1)
            p_i = p_i_1
            print(p_i_1)
        else: 
            if m_i_2 in ascii_bytes:
                decrypt_data_arr.append(m_i_2)
                p_i = p_i_2
                print(p_i_2)
            else:
                print("Error???????????????????")
    txt = ''.join([chr(s) for s in decrypt_data_arr])
    # print(txt)
    # print(decrypt_data_arr)
    rwf.write_data_1byte('one_time_pad_wrong_decrypt.txt', decrypt_data_arr)
        

In [50]:
one_time_pad_wrong()

00000000 00000000 0
0 90 14
14 84 <class 'int'> 14
90
01011010 01011010 90
90 180 220
134 104 <class 'int'> 220
180
10110100 00101101 45
45 135 72
101 207 <class 'int'> 72
45
00101101 10110100 180
180 14 148
32 154 <class 'int'> 148
180
10110100 00101101 45
45 135 93
112 218 <class 'int'> 93
45
00101101 10110100 180
180 14 124
200 114 <class 'int'> 124
14
00001110 01110000 112
112 202 31
111 213 <class 'int'> 31
112
01110000 00001110 14
14 104 10
4 98 <class 'int'> 10
104
01101000 00010110 22
22 112 28
10 108 <class 'int'> 28
112
01110000 00001110 14
14 104 13
3 101 <class 'int'> 13
104
01101000 00010110 22
22 112 29
11 109 <class 'int'> 29
112
01110000 00001110 14
14 104 125
115 21 <class 'int'> 125
14
00001110 01110000 112
112 202 234
154 32 <class 'int'> 234
202
11001010 01010011 83
83 173 60
111 145 <class 'int'> 60
83
01010011 11001010 202
202 36 172
102 136 <class 'int'> 172
202
11001010 01010011 83
83 173 141
222 32 <class 'int'> 141
173
10101101 10110101 181
181 15 214
99 217 <

Задание 1. 
Расшифровать файл im3_hill_c_all.bmp. Ключ – матрица
K=[[189 58]
[ 21 151]].

In [51]:
def prepareData(data):
    vectors = []
    for i in range(0, len(data), 2):
        vector = data[i:i+2] 
        vectors.append(vector)
    vectors_array = np.array(vectors, dtype=object)
    return vectors_array


def mod_inverse(a, m):
    a = a % m
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None


def matrix_inverse_mod_l(matrix, l):
    det_matrix = round(np.linalg.det(matrix))
    if det_matrix == 0:
        return None
    det_matrix_modL = mod_inverse(det_matrix, l)
    x1 = [matrix[1][1] % l * det_matrix_modL, -matrix[0][1] % l * det_matrix_modL]
    x2 = [-matrix[1][0] % l * det_matrix_modL, matrix[0][0] % l * det_matrix_modL]
    matrix_rev = np.array([x1, x2])
    return matrix_rev

In [52]:
def ex_1():
    K = np.array([[189, 58], [21, 151]])
    l = 256
    encrypt_data = rwf.read_data_1byte('im3_hill_c_all.bmp')
    data_arrs = prepareData(encrypt_data)
    # print('data', data_arrs)
    decrypt_data = []
    A_rev = matrix_inverse_mod_l(matrix=K, l=l)
    for mtx in data_arrs:
        P = np.dot(A_rev, mtx)
        for el in P:
            num = round(el % l)
            decrypt_data.append(num)
            # print("num =", num)
    rwf.write_data_1byte('im3_hill_c_all_decrypt.bmp', decrypt_data)

In [53]:
#mod_inverse(27321, 256)
ex_1()

Задание 2. 
Расшифровать файл m18_hill_c_all.bmp. ШифрХилла.K=[[ 47 239] [119
108]]. Зашифровать, оставив первые 50 байт без изменения.

In [54]:
def ex_2():
    K = np.array([[47, 239], [119, 108]])
    l = 256
    encrypt_data = rwf.read_data_1byte('m18_hill_c_all.bmp')
    data_arrs = prepareData(encrypt_data)
    decrypt_data = []
    A_rev = matrix_inverse_mod_l(matrix=K, l=l)
    for mtx in data_arrs:
        P = np.dot(A_rev, mtx)
        for el in P:
            num = round(el % l)
            decrypt_data.append(num)
    rwf.write_data_1byte('m18_hill_c_all_decrypt.bmp', decrypt_data)

    data_arr = rwf.read_data_1byte('m18_hill_c_all_decrypt.bmp')
    data_arrs = prepareData(data_arr[50:]) 
    crypt_arr = []
    for i in range(50):
        crypt_arr.append(data_arr[i])
    for mtx in data_arrs:
        print('mtx', mtx)
        C = np.dot(K, mtx)
        print('C', C)
        for el in C:
            print('el', el)
            num = round(el % l)
            crypt_arr.append(num)
    rwf.write_data_1byte('m18_hill_c_all_crypt50.bmp', crypt_arr)


In [55]:
ex_2()

mtx [0 0]
C [0 0]
el 0
el 0
mtx [0 0]
C [0 0]
el 0
el 0
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el 72930
el 57885
mtx [255 255]
C [72930 57885]
el

Задание 3.
Дешифровать файл p1_hill_c_all.png. ШифрХилла. Первые четыре байта в любом
png-файле: 137, 80, 78, 71.

In [56]:
# import itertools
# import multiprocessing

# def is_decrypt_png(matrix, l, data_arrs, result_queue):
#     decrypt_data = []
#     det_matrix = round(np.linalg.det(matrix))
#     if det_matrix == 0:
#         return False
#     det_matrix_modL = mod_inverse(det_matrix, l)
#     x1 = [matrix[1][1] % l * det_matrix_modL, -matrix[0][1] % l * det_matrix_modL]
#     x2 = [-matrix[1][0] % l * det_matrix_modL, matrix[0][0] % l * det_matrix_modL]
#     matrix_rev = np.array([x1, x2])
#     for mtx in data_arrs:
#         P = np.dot(matrix_rev, mtx)
#         for el in P:
#             num = round(el % l)
#             decrypt_data.append(num)
#     if (decrypt_data[0] == 137 and decrypt_data[1] == 80 and decrypt_data[2] == 78 and decrypt_data[3] == 71):
#         result_queue.put(matrix)  # Отправляем матрицу в очередь
#         return True
#     return False

# def find_key():
#     read_data = rwf.read_data_1byte('p1_hill_c_all.png')
#     png_data = prepareData(read_data[:4])
#     l = 256
#     upper_limit = 300
#     keys = itertools.product(range(upper_limit + 1), repeat=4)

#     result_queue = multiprocessing.Queue()  # Создаем очередь для результатов

#     with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
#         # Используем map для передачи аргументов
#         pool.starmap(is_decrypt_png, 
#                       [([[a, b], [c, d]], l, png_data, result_queue) for a, b, c, d in keys])

#     # Проверяем, есть ли результаты в очереди
#     while not result_queue.empty():
#         matrix = result_queue.get()
#         print("Найдена матрица:", matrix)
#         return matrix()

# Вызов функции


In [57]:
def find_key_for_png(crypto_data, true_data):
    c = np.array(crypto_data, dtype=np.float64).T
    m = np.array(true_data, dtype=np.float64).T
    l = 256
    m_rev = matrix_inverse_mod_l(matrix=m, l=l)
    k = np.dot(c, m_rev)
    key = k % 256
    key_arr = np.array(key, dtype=np.float64)
    print("Found key:", key_arr)
    return key_arr

In [58]:
def ex_3():
    l = 256
    encrypt_data = rwf.read_data_1byte('p1_hill_c_all.png')
    crypto_data = prepareData(encrypt_data[:4])
    png_data = np.array([[137, 80], [78, 71]])
    K = find_key_for_png(crypto_data=crypto_data, true_data=png_data)
    data_arrs = prepareData(encrypt_data)
    decrypt_data = []
    A_rev = matrix_inverse_mod_l(matrix=K, l=l)
    for mtx in data_arrs:
        P = np.dot(A_rev, mtx)
        for el in P:
            num = round(el % l)
            decrypt_data.append(num)
    rwf.write_data_1byte('p1_hill_c_all_decrypt.png', decrypt_data)

In [59]:
ex_3()

Found key: [[103. 239.]
 [ 43.  32.]]


Задание 4.
Дешифровать png-файл b4_hill_c_all.png. Первые четыре байта в любом
png-файле: 137, 80, 78, 71.

In [60]:
def ex_4():
    l = 256
    encrypt_data = rwf.read_data_1byte('b4_hill_c_all.png')
    crypto_data = prepareData(encrypt_data[:4])
    png_data = np.array([[137, 80], [78, 71]])
    K = find_key_for_png(crypto_data=crypto_data, true_data=png_data)
    data_arrs = prepareData(encrypt_data)
    decrypt_data = []
    A_rev = matrix_inverse_mod_l(matrix=K, l=l)
    for mtx in data_arrs:
        P = np.dot(A_rev, mtx)
        for el in P:
            num = round(el % l)
            decrypt_data.append(num)
    rwf.write_data_1byte('b4_hill_c_all_decrypt.png', decrypt_data)

In [61]:
ex_4()

Found key: [[175. 251.]
 [ 75. 214.]]


Задание 5.
Дешифровать файл text2_hill_c_all.txt. Известно, что текст в файле
начинается со слова Whose.

In [64]:
def ex_5():
    l = 256
    encrypt_data = rwf.read_data_1byte('text2_hill_c_all.txt')
    crypto_data = prepareData(encrypt_data[:4])
    word = "Whos"
    word_data = prepareData([ord(s) for s in word])
    print (crypto_data, word_data)
    K = find_key_for_png(crypto_data=crypto_data, true_data=word_data)
    data_arrs = prepareData(encrypt_data)
    decrypt_data = []
    A_rev = matrix_inverse_mod_l(matrix=K, l=l)
    for mtx in data_arrs:
        P = np.dot(A_rev, mtx)
        for el in P:
            num = round(el % l)
            decrypt_data.append(num)
    rwf.write_data_1byte('text2_hill_c_all_decrypt.txt', decrypt_data)

In [66]:
ex_5()

[[15 169]
 [93 158]] [[87 104]
 [111 115]]
Found key: [[ 89. 162.]
 [ 23. 135.]]
